# Converting data into the interspike interval (ISI) format

In order to use our package, the spike data has to be converted into the ISI format. In this format for each trial the data contains two 1D numpy arrays. The first array contains the ISIs of all of the neurons, starting from the time interval between the trial start and the first spike, and ending with the time interval between the last spike and the trial end. The second array contains the corresponding neuron indices (ids), which are integers that start from 0. End of trial is marked with -1.

**ISI format**: the data is a numpy array of size `(num_trial,2)`, `dtype = np.ndarray`, where `data[i,0]` contains all of the ISIs for trial `i`, and `data[i,0]` contains the corresponding neuronal IDs.

**Example of the ISI format**:  For example, consider the data with only one trial. The 1st neuron (id=0) spike times are `0.12, 0.15, 0.25`. The second neuron (id=1) spike times are `0.05, 0.2`. Trial 0 starts at `t=0`, and ends at `t=0.27`. 

In the ISI format, the first ISI is between the trial start time and the first spike, which is $0.05$, and the first spike was emitted by a neuron with `id=1`. The second ISI is between $t=0.05$ and $t=0.12$, and the second spike was emitted by a neuron with `id=0`. The last ISI is between the last spike `t=0.25` and the trial end time `t=0.27`, and it is marked with `-1`. The data in the ISI format will be an array of size `(1,2)`, where `data_ISI[0,0]=np.array([0.05,0.07,0.03,0.05,0.05,0.02])`, and `data_ISI[0,1]=np.array([1,0,0,1,0,-1])`. 

**In this task you will convert the data from the spike times format to the ISI format**. You will upload the data in the spike format: `spike_data` is a dictionary with two keys: `trial_end_time`, and `spikes`. The first key is a 1D array of length `num_trial` with the recorded trial end times (assume that all trials start from `t=0`), where `num_trial` is the number of trials. The second key is a numpy array of size `(num_neuron,num_trial)`, where each element is a 1D array of spike times for a neuron on a given trial.

For the example above, the data in this format will be: `data_spikes=[{"trial_end_time": np.array([0.27]),"spikes": np.array([np.array([0.12, 0.15, 0.25]), np.array([0.05, 0.2])])`. 

In [1]:
# Package installation - needed to run in google Collab. Skip this cell if you use local jupyter notebook
!pip install git+https://github.com/engellab/neuralflow

#Make data folder and download the data in spike times format.
!mkdir data
import urllib.request
urllib.request.urlretrieve('https://github.com/engellab/neuralflow/raw/master/tutorials/CCN2021/data/Part1_spike_times_data.pkl', "data/Part1_spike_times_data.pkl")

  Cloning https://github.com/engellab/neuralflow to /private/var/folders/xt/nql8j4c959l6b4s54h4l91340000gn/T/pip-req-build-ud1_4e1v
  Running command git clone -q https://github.com/engellab/neuralflow /private/var/folders/xt/nql8j4c959l6b4s54h4l91340000gn/T/pip-req-build-ud1_4e1v
  Created wheel for neuralflow: filename=neuralflow-2.0.1-cp38-cp38-macosx_10_9_x86_64.whl size=113033 sha256=933fa431cbb6d38ccf8385cae21ebf6075e0cf8acc2dc481d379aa17e328da26
  Stored in directory: /private/var/folders/xt/nql8j4c959l6b4s54h4l91340000gn/T/pip-ephem-wheel-cache-7o0cjw2f/wheels/ed/ef/2f/a174789969deebfdb593975a885eafc637a5af35a925ac34ff
Successfully built neuralflow
  Attempting uninstall: neuralflow
    Found existing installation: neuralflow 2.0.1
    Uninstalling neuralflow-2.0.1:
      Successfully uninstalled neuralflow-2.0.1
mkdir: data: File exists


('data/Ex1.pkl', <http.client.HTTPMessage at 0x110c2f640>)

In [1]:
# Import packages 
import neuralflow
import numpy as np
import matplotlib.pyplot as plt, matplotlib.gridspec as gridspec
import scipy
import pickle

In [2]:
# Cell 1 - Load the data in the spike format and print it

with open("data/Part1_spike_times_data.pkl","rb") as fp:
    spike_data = pickle.load(fp)

# Extract the spike times and trial end times
spike_times = spike_data['spikes']
trial_ends = spike_data['trial_end_time']
    
print(f'Spike data contains {spike_times.shape[0]} neurons and {spike_times.shape[1]} trials.')
print(f'Trial 3 starts at t=0, and ends at t={trial_ends[3]}')
print(f'At trial 3, the spike times of a neuron with id=0 are: {spike_times[0,3]}') 
      
######INSERT YOUR CODE HERE############
print(f'At trial 3, the spike times of a neuron with id=1 are: {None}')     
#######################################

Spike data contains 3 neurons and 5 trials.
Trial 3 starts at t=0, and ends at t=0.1278
At trial 3, the spike times of a neuron with id=0 are: [0.00202557 0.01655405 0.0216746  0.02636602 0.0494601  0.06425032
 0.0672514  0.07441427 0.091588   0.09882371 0.09994917 0.10403823
 0.10716863 0.11026739 0.12120375 0.12156251]
At trial 3, the spike times of a neuron with id=1 are: None


Our `EnergyModel` class offers a method that converts the spike data into the ISI format. 

`data = EnergyModel.transform_spikes_to_isi(spikes, time_epoch, **kwargs)`

Here the inputs are: 

`spikes: a 2D array with all of the spike times, the shape is (num_neuron,num_trial), and each element is a 1D array of spike times.`
    
`time_epoch`: list of tuples, one for each trial. Each tuple consists of two elements: trial start time and trial end time.
    
The output is the data in the ISI format. See the additional optional parameters in the doc string of this function.





In [3]:
# Cell 2 - Convert the spike data into the ISI format. 

# Initialize an instance of the EnergyModel class
em = neuralflow.EnergyModel()

# Create the time_epoch list
time_epoch = [(0,trial_end) for trial_end in trial_ends]

# Convert the data into the ISI format
data_ISI=em.transform_spikes_to_isi(spike_times, time_epoch)

# Let us print the data in the ISI format
print(f'The ISI data contains {data_ISI.shape[0]} trials.')
print(f'At trial 3, the ISIs from all of the neurons are {data_ISI[3,0]}')
print(f'At trial 3, the corresponding neuron ids are {data_ISI[3,1]}')

# We can calculate the sorted spike times of all neurons at a particular trial using np.cumsum function.

######INSERT YOUR CODE HERE############
print(f'At trial 3, the sorted spike times of all neurons are: {None}') 
#######################################

The ISI data contains 5 trials.
At trial 3, the ISIs from all of the neurons are [0.00202557 0.01181484 0.00271364 0.00512054 0.00469143 0.02309408
 0.01479022 0.00300108 0.0010751  0.00213459 0.00395319 0.01287632
 0.00429741 0.0003679  0.00482648 0.00204132 0.00112546 0.00408906
 0.0031304  0.00309876 0.01093637 0.00035876 0.00219539 0.0040421 ]
At trial 3, the corresponding neuron ids are [ 0  1  0  0  0  0  0  0  1  1  0  1  0  1  1  0  0  0  0  0  0  0  1 -1]
At trial 3, the sorted spike times of all neurons are: None
